# A CNN for cell fate classification of hematopoietic stem cells

A short tutorial/redo of the analysis done in the following paper: 

[Prospective identification of hematopoietic lineage choice by deep learning](http://www.nature.com/nmeth/journal/v14/n4/full/nmeth.4182.html)    
Felix Buggenthin, Florian Buettner, Philipp S Hoppe, Max Endele, Manuel Kroiss, Michael Strasser, Michael Schwarzfischer, Dirk Loeffler, Konstantinos D Kokkaliaris, Oliver Hilsenbeck, Timm Schroeder, Fabian J Theis, Carsten Marr; *Nature Methods 14, 403–406* (2017)


[Code](https://github.com/QSCD/HematoFatePrediction)



In [ ]:
import keras
import numpy as np
from talk_utils import tile_raster_images
%matplotlib inline
import matplotlib.pyplot as plt

## construct the keras network
- Conv1
- Conv2
- Conv3
- merge with speed
- fc6
- fc7
- fc8/softmax

![HematoCNN](images/hemato_cnn.png)

In [ ]:
from talk_hemato_utils import create_hemato_cnn
CNN = create_hemato_cnn()
CNN.summary()

load the pretrained weights onto the model

In [ ]:
import caffe2keras_conversion
CNN = caffe2keras_conversion.load_weights_caffe2keras('../pretrained_hemato_net.hdf5', CNN, bn_trainable=True, other_param_trainable=True)

# loading the retrained weights and the corresponding data
Turns out that the pretrained weights are hard to transfer to keras (Batch normalization etc). Hence, I retrained the network on a subset of the data provided in `images_round3_test_annotated.pickle`. 
<img src="images/latent_cells.png" alt="latent cells" style="width: 600px;"/>

**Note**: This is **VERY different** from the "across-movie" training/prediction done in the original paper.
I instead train and evaluate on the same experiment here (samples in train/testset are still disjunct, but come from the same experiment).
Therefore the results in this notebook are **overoptimistic**. But this notebook serves for demonstration only anyways :)
![roundRobin](images/roundrobin.png)

In [ ]:
CNN.load_weights('../retrained.h5')

we need to get the exact same datasplit I used from training, such that our testset is different from the training

In [ ]:
import pickle
with open('../retrained_datasplit.pickle', 'rb') as fh:
    X_train,X_val,X_test,\
    y_train,y_val,y_test,\
    mov_train, mov_val, mov_test,\
    cell_train, cell_val, cell_test = pickle.load(fh)

print(" %d train data\n %d val. data\n %d test data" % (len(X_train), len(X_val), len(X_test)))

for fast computation, restrict to just 10000 test-samples

In [ ]:
X_test   = X_test[:10000]
y_test   = y_test[:10000]
mov_test = mov_test[:10000]
cell_test= cell_test[:10000]

# A first look at the data
Let's look at a few examples of the two classes; 
- they look very similar to non-experts
- sometimes differentiated cells can be observed, which are distinct (e.g. megakaryocytes)

In [ ]:
X_class0 = X_test[y_test[:,1]==0]
X_class1 = X_test[y_test[:,1]==1]

tile_raster_images(X_class0[:1000,:,:,0], img_dim=(1,2), 
                   tile_shape=(20,50), scale_rows_to_unit_interval=False, figsize=(20,20))

tile_raster_images(X_class1[:1000,:,:,0], img_dim=(1,2), 
                   tile_shape=(20,50), scale_rows_to_unit_interval=False, figsize=(20,20))

in addition we have the movement speed as a feature. Note that speed was already standardized (mean=0, std=1), hence the negative values

In [ ]:
plt.hist(mov_test[y_test[:,1]==0], bins = np.linspace(-1,10,100), histtype='step', normed=True);
plt.hist(mov_test[y_test[:,1]==1], bins = np.linspace(-1,10,100), histtype='step', normed=True);
# plt.hist(mov_test[y_test[:,1]==1], 100);
plt.xlabel('Movement speed')
plt.ylabel('Relative frequency')
plt.legend(['Class1', 'Class2']);

# Prediction of annotated cells



## Single image prediction
- Predict all 10000 samples from the testset (takes ~10sec) 
- look at the histogram of the scores.
- what is the accuracy/confusion matrix
- what is the area under the ROC curve

**Hint**: 
- the model has to inputs, the image and speed. Feed them into the model as a list
- `sklearn.metrics` has implementations of confusion/AUC already!

In [ ]:
from sklearn.metrics import accuracy_score, auc, confusion_matrix
yhat = CNN.predict([X_test, mov_test], batch_size=128)

In [ ]:
plt.hist(yhat[:,1],100);
plt.xlabel('Prediction Score');
plt.ylabel('Frequency');

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(np.argmax(yhat,1), np.argmax(y_test, 1)))

from talk_utils import plot_confusion_matrix
plot_confusion_matrix(yhat,  y_test, classes=[0,1]);

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(np.argmax(y_test, 1), yhat[:,1], drop_intermediate=False)
the_auc = auc(fpr, tpr)
print(the_auc)

plt.figure()
plt.scatter(fpr, tpr, c=thresholds, cmap=plt.cm.bwr);
plt.plot(fpr,tpr,'k');

In [ ]:
from talk_hemato_utils import get_auc
get_auc(yhat[:,1], y_test[:,1], do_plot=True)

## Aggregate cells over multiple timepoints
So far, we predict a class for each single cell patch. However, through tracking, we can pool over image patches that belong to the same cell. That should make the classification more robust.
There's a couple of ways to aggregate:
- **hard voting**: each sample is first classified (into 0,1) and then we take the average: I.e 80% of the samples of the cell were classified as 1 -> vote for class 1 
- **soft voting**: we could also first average the class scores of all samples of the same cell, then discretize into [0,1]

- use a **neural network** to do the aggregation. [Buggenthin et al.] use a RNN to also incorporate the time dependence of the images. (We skip this for simplicity)


**Note**: We're somewhat cheating here: The train/val/test split was agnostic of the image patches being linked together. For example, cell 1 could have 10 patches in the training set, and 20 patches in the test set -> **some leakage from training to test** if the patches are strongly correlated and our results are overoptimistic. 

In [Buggenthin et al.], the training/validation/testsets are **different experiments** to avoid this and similar issues!

In [ ]:
cellid = 353
plt.plot(yhat[cell_test==cellid, 1]); 
plt.xlabel('image patch')
plt.ylabel('lineage score')
plt.title('Cell %d with true label %d' % (cellid, y_test[cell_test==cellid,1][0]))
plt.show()

let pandas do the aggregation magic..,

In [ ]:
import pandas as pd

df = pd.DataFrame(np.hstack([y_test, yhat, cell_test[:,np.newaxis]]), columns=['y0', 'y1', 'score0', 'score1', 'cellid'])
df['yhat'] = (df['score1'] > 0.5).values.astype('int')
df.head()

In [ ]:
aggr = df.groupby('cellid').mean()
aggr = aggr.rename(columns={'score1': 'softvote', 'yhat': 'hardvote'}) 
aggr.head()

In [ ]:
# to comply with the usual two column scores
softvoted_yhat = np.vstack([1-aggr['softvote'], aggr['softvote']]).T
hardvoted_yhat = np.vstack([1-aggr['hardvote'], aggr['hardvote']]).T
voted_y = np.vstack([aggr['y0'], aggr['y1']]).T

putting that all together

In [ ]:
def aggregate_cell_scores(df):
    "aggregates all scores from the same cell, doing either soft or hard voting"
    df['yhat'] = (df['score1'] > 0.5).values.astype('int')
    aggr = df.groupby('cellid').mean()
    aggr = aggr.rename(columns={'score1': 'softvote1', 'yhat': 'hardvote1'})
    aggr['softvote0'] = 1-aggr['softvote1']
    aggr['hardvote0'] = 1-aggr['hardvote1']

    return aggr

### Soft voting

In [ ]:
aggr = aggregate_cell_scores(df)
aggr.head()

In [ ]:
plt.figure()
plot_confusion_matrix(aggr[['softvote0', 'softvote1']].values,  aggr[['y0', 'y1']].values, classes=[0,1]);
get_auc(aggr['softvote1'].values, aggr['y1'], do_plot=True)

### Hard voting

In [ ]:
plt.figure()
plot_confusion_matrix(aggr[['hardvote0', 'hardvote1']].values,  aggr[['y0', 'y1']].values, classes=[0,1]);
get_auc(aggr['hardvote1'].values, aggr['y1'], do_plot=True)

# Applying the CNN to "latent" cells
So far, we trained the NN on cells that expressed some cell fate markers (hence they were already differentiated).
The most important contribution of [Buggenthin et al.] is that this classifier can also be applied to cells expressing no marker yet (latent), and still correctly predict what will happen in the future (we know the future via tracking)

<img src="images/latent_cells.png" alt="latent cells" style="width: 600px;"/>


In [ ]:
from talk_hemato_utils import load_data
X_l, y_l, movement_l, cellIDs_l = load_data('../images_round3_test_latent.pickle', N=None, randomize=True)
print("%d patches, %d cells in total" % (len(X_l), len(np.unique(cellIDs_l))))

put the "meta"-data into a pandas dataframe. This will become very handy later

In [ ]:
df = pd.DataFrame(np.hstack([y_l, cellIDs_l[:,np.newaxis]]), columns=['y0', 'y1', 'cellid'])

load the 'inverted generation', i.e. the position of the cell wrt to marker onset in the tree. inv.gen=-2 means that the cell fate marker will turn on two generations downstream of that cell

## Loading the inverted generations

In [ ]:
import h5py
mat = h5py.File('../anno_file.h5', 'r')
gens = mat['anno']['latent'][:][0]
gens = np.array([gens[i] for i in cellIDs_l])  # expand into a vector, one element for each patch
df['gens'] = gens

In [ ]:
df.head()

## select a smaller subset of cells
for computational reasons again, restrict to a few hundreds of cells

In [ ]:
from talk_hemato_utils import ismember

Ncells = 200
cell_subset = np.random.choice(np.unique(cellIDs_l), Ncells, replace=False)  # lets take 100 cells and all their patches
subset_ix = ismember(cellIDs_l, cell_subset)
subset_ix = np.array([_ is not None for _ in subset_ix])

X_l = X_l[subset_ix]
y_l = y_l[subset_ix]
movement_l = movement_l[subset_ix]
df = df.iloc[subset_ix]

print("%d patches, %d cells in total" % (len(X_l), len(np.unique(df['cellid']))))

## Prediction of latent cells
Takes 2min on a Quadcore

In [ ]:
%time yhat_l = CNN.predict([X_l, movement_l], batch_size=128, verbose=1)

df['score0'] = yhat_l[:,0]  # put the predictions into the dataframe
df['score1'] = yhat_l[:,1]

## Aggregation
again, we want to aggregate the scores of patches belonging to the same cell.

What's the distribution of patch scores for a single cell:

In [ ]:
cellid = cell_subset[0]
df_cell = df.query("cellid==@cellid")
plt.hist(df_cell['score1'],100);
plt.title('Cell %d (%d patches) with true label %f' % (cellid, len(df_cell),df_cell.y1.values[0]))
plt.xlabel('lineage score');

aggregate as before

In [ ]:
aggr_l = aggregate_cell_scores(df)
aggr_l.head(10)

## Evaluation / AUC
Calculate the AUCs for raw patches and the aggregated cells

In [ ]:
get_auc(df['score1'], df['y1'], do_plot=True)  # without aggregation
get_auc(aggr_l['softvote1'], aggr_l['y1'], do_plot=True);
get_auc(aggr_l['hardvote1'], aggr_l['y1'], do_plot=True);

Finally, let's check how our predictive performance changes as we move further away from the observed marker onset.

<img src="images/latent_cells.png" alt="latent cells" style="width: 600px;"/>


In [ ]:
# split into the different generations relative to onset
auc_list = []
for g in [-5,-4,-3,-2,-1]:
    df_g = aggr_l.query('gens==@g')
    auc = get_auc(df_g['softvote1'], df_g['y1'])
    auc_list.append(auc)


plt.plot([-5,-4,-3,-2,-1], auc_list, marker='x')
plt.xlabel('inverted generation');
plt.ylabel('AUC');

**Note**: Again, keep in mind that we didnt evaluate on a totally different testset (I trained the classifier on a subset of the "annotated" cells in a single experiment). So applying the classifier to "latent" cells of the same movie might be **too optimistic** (e.g. overfitting on lighting conditions etc).